In [1]:
import pandas as pd
import sqlite3

# Load the dataset into a Pandas DataFrame
print("Loading dataset into df")
df = pd.read_csv('job_descriptions.csv')
print("Finished loading dataset")

Loading dataset into df
Finished loading dataset


In [2]:
conn = sqlite3.connect('job.db')

# Get all column names from the DataFrame
columns = df.columns.tolist()
df.columns = df.columns.str.replace(' ', '_')

# Create a table with FTS5 including all columns from the DataFrame
columns_definitions = ', '.join([str(col).replace(" ","_") for col in columns])  # FTS5 requires text columns
conn.execute(f'''
CREATE VIRTUAL TABLE IF NOT EXISTS my_table USING fts5(
    {columns_definitions}
);
''')

# Insert data into the table
df.to_sql('my_table', conn, if_exists='replace', index=False)

1615940

In [23]:
# def search_database(conn, columns, keywords, limit=10):
#     """ 
#     Search the SQLite database using Full-Text Search (FTS5).
    
#     :param conn: SQLite connection object 
#     :param columns: List of column names to search 
#     :param keywords: List of keywords to search for 
#     :param limit: Number of rows to return 
#     :return: List of matching rows 
#     """
#     if not keywords:
#         return []  # Return empty if no keywords provided

#     # Create a MATCH clause for each column
#     columns_match = ' OR '.join([f'{col} MATCH ?' for col in columns])

#     # Combine keywords into one search string for MATCH clause
#     #keywords_query = ' AND '.join([' '.join(keywords)])  # Note: Adjust based on the search logic needed.
#     keywords_query = 'Python Java'
#     # Prepare the SQL query
#     #query = f"SELECT * FROM my_table WHERE ({columns_match}) AND ({keywords_query}) LIMIT ?;"
#     query = f"SELECT * FROM my_table WHERE ({columns_match}) LIMIT ?;"

#     # Prepare parameters
#     parameters = [keywords_query] * len(columns) + [limit]

#     # Print debug information to check the query and parameters
#     print("Query:", query)
#     print("Parameters:", parameters)

#     # Execute the query
#     cursor = conn.execute(query, parameters)

#     # Fetch and return the results
#     results = cursor.fetchall()
#     return results
def search_database(conn, columns, keywords, limit=10):
    """ 
    Search the SQLite database using Full-Text Search (FTS5).
    
    :param conn: SQLite connection object 
    :param columns: List of column names to search 
    :param keywords: List of keywords to search for 
    :param limit: Number of rows to return 
    :return: List of matching rows 
    """
    
    if not keywords:
        return []  # Return empty if no keywords provided

    # Create a space-separated string for the keywords
    keywords_query = ' '.join(keywords)  # "Python Java"

    # Create a MATCH clause for each column using the same keywords
    columns_match = ' OR '.join([f"{col} MATCH ?" for col in columns])

    # Prepare the SQL query
    query = f"SELECT * FROM my_table WHERE {columns_match} LIMIT ?;"

    # Prepare parameters: the keywords query repeated for each column plus the limit
    parameters = [keywords_query] * len(columns) + [limit]  # List is now size of (columns + 1)

    # Print debug information to check the query and parameters
    print("Query:", query)
    print("Parameters:", parameters)

    # Execute the query
    cursor = conn.execute(query, parameters)

    # Fetch and return the results
    results = cursor.fetchall()
    return results

In [24]:
columns_to_search = ['Job_Description', "skills"]  # All columns
keywords = ['Python', 'Java']  # Replace with the keywords you want to search for

results = search_database(conn, columns=columns_to_search, keywords=keywords, limit=10)

# Display the results
for row in results:
    print(row)

Query: SELECT * FROM my_table WHERE Job_Description MATCH ? OR skills MATCH ? LIMIT ?;
Parameters: ['Python Java', 'Python Java', 10]


OperationalError: unable to use function MATCH in the requested context